<a href="https://colab.research.google.com/github/gunisha30/codes/blob/master/Copy_of_audio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install praat-parselmouth

In [0]:
!sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!sudo apt-get install ffmpeg libav-tools
!sudo pip install pyaudio

In [0]:
import librosa as l
import os
import numpy as np
os.chdir('/content/drive/My Drive/awarathon')
arr,sr=l.load('George.wav')
arr.size#/sr #length
#print(sr)
2871072/22050
u=22049*5
l=u-22049
arrnew=arr[l:u+1]
np.sum(arrnew)/22050

In [0]:
#convert mp4 to wav 
import sys
import os
from moviepy.editor import *

# name is the path, if its in the root directory then its just the video name
def convert_to_audio(name):
  video = VideoFileClip(name)
  audio = video.audio
  audio.write_audiofile('snigdhaq3.wav')

#folder renaming: first-Nishkarsh second-Yash third-Vishesh

#os.chdir('/content/drive/My Drive/awarathon/videos')
name="snigdhaq3.mp4"
convert_to_audio(name)

In [0]:
import parselmouth
from parselmouth.praat import call, run_file
import inspect as i
import sys
import scipy.io.wavfile
import math
import moviepy.editor as mp

#os.chdir('/content')
def mysppaus(m,p,l):              #pause rate and articulation rate
    sound=p+"/"+m+".wav"          #entire path till audio file
    sourcerun="/content/drive/My Drive/awarathon/myspsolution.praat"                  #path till mysolution.praat
    path=p                        #path till the folder which has audio file
    try:
        objects= run_file(sourcerun, -20, 2, 1, "yes",sound,path, 80, 400, 0.01, capture_output=True) 
        z1=str(objects[1])
        z2=z1.strip().split()
        print(z2)
        print ("articulation rate=",z2[3])
        p=int(z2[1])                          #pause length in secs
        #p=p/60                                #pause length in mins
        a=p/l                                 #ratio of pause length and total length
        print("rate of pause=",a)
        print ("pause percentage=",a*100)
        #print(p)
    except:
        print ("Try again the sound of the audio was not clear")
    return;


audioname="anjaliq6"
path1="/content/drive/My Drive/awarathon/videos"
#path1='/content'
#length of video files 

clip = mp.VideoFileClip('/content/drive/My Drive/awarathon/videos/anjaliq6.mp4')
duration = math.floor(clip.duration)
image_time_interval = 2
#duration=duration/60
print("length=",duration/60)

mysppaus(audioname,path1,duration)

In [0]:
pip install aubio

In [0]:
import scipy.io.wavfile
import os
os.chdir('/content/drive/My Drive/awarathon')
rate, arr = scipy.io.wavfile.read("George.wav")
print(arr.shape[0])
print(arr.size)
length=arr.shape[0]/rate
print(length/60)
  


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
print(os.ch())
#!pip install youtube_dl

In [0]:
import youtube_dl

def download_from_vimeo(vid_link):
    ydl = youtube_dl.YoutubeDL({'outtmpl':'vid.mp4'})

    with ydl:
        result = ydl.extract_info(
            vid_link,
            download=True # We just want to extract the info
        )

    if 'entries' in result:
        # Can be a playlist or a list of videos
        video = result['entries'][0]
    else:
        # Just a video
        video = result

    print(video)

download_from_vimeo('https://player.vimeo.com/video/348632880')

In [2]:
import sys
from aubio import source, pitch

# if len(sys.argv) < 2:
#     print("Usage: %s <filename> [samplerate]" % sys.argv[0])
#     sys.exit(1)
# filename = sys.argv[1]
filename="/content/drive/My Drive/awarathon/audio1.wav"
downsample = 1
samplerate = 44100 // downsample
# if len( sys.argv ) > 2: samplerate = int(sys.argv[2])


win_s = 4096 // downsample # fft size
hop_s = 512  // downsample # hop size

s = source(filename, samplerate, hop_s)
samplerate = s.samplerate

tolerance = 0.8

pitch_o = pitch("yin", win_s, hop_s, samplerate)
pitch_o.set_unit("midi")
pitch_o.set_tolerance(tolerance)

pitches = []
confidences = []

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    pitch = pitch_o(samples)[0]
    #pitch = int(round(pitch))
    confidence = pitch_o.get_confidence()
    #if confidence < 0.8: pitch = 0.
    print("%f %f %f" % (total_frames / float(samplerate), pitch, confidence))
    pitches += [pitch]
    confidences += [confidence]
    total_frames += read
    if read < hop_s: break

if 0: sys.exit(0)

#print pitches
import os.path
from numpy import array, ma
import matplotlib.pyplot as plt
from demo_waveform_plot import get_waveform_plot, set_xlabels_sample2time

skip = 1

pitches = array(pitches[skip:])
confidences = array(confidences[skip:])
times = [t * hop_s for t in range(len(pitches))]

fig = plt.figure()

ax1 = fig.add_subplot(311)
ax1 = get_waveform_plot(filename, samplerate = samplerate, block_size = hop_s, ax = ax1)
plt.setp(ax1.get_xticklabels(), visible = False)
ax1.set_xlabel('')

def array_from_text_file(filename, dtype = 'float'):
    filename = os.path.join(os.path.dirname(__file__), filename)
    return array([line.split() for line in open(filename).readlines()],
        dtype = dtype)

ax2 = fig.add_subplot(312, sharex = ax1)
ground_truth = os.path.splitext(filename)[0] + '.f0.Corrected'
if os.path.isfile(ground_truth):
    ground_truth = array_from_text_file(ground_truth)
    true_freqs = ground_truth[:,2]
    true_freqs = ma.masked_where(true_freqs < 2, true_freqs)
    true_times = float(samplerate) * ground_truth[:,0]
    ax2.plot(true_times, true_freqs, 'r')
    ax2.axis( ymin = 0.9 * true_freqs.min(), ymax = 1.1 * true_freqs.max() )
# plot raw pitches
ax2.plot(times, pitches, '.g')
# plot cleaned up pitches
cleaned_pitches = pitches
#cleaned_pitches = ma.masked_where(cleaned_pitches < 0, cleaned_pitches)
#cleaned_pitches = ma.masked_where(cleaned_pitches > 120, cleaned_pitches)
cleaned_pitches = ma.masked_where(confidences < tolerance, cleaned_pitches)
ax2.plot(times, cleaned_pitches, '.-')
#ax2.axis( ymin = 0.9 * cleaned_pitches.min(), ymax = 1.1 * cleaned_pitches.max() )
#ax2.axis( ymin = 55, ymax = 70 )
plt.setp(ax2.get_xticklabels(), visible = False)
ax2.set_ylabel('f0 (midi)')

# plot confidence
ax3 = fig.add_subplot(313, sharex = ax1)
# plot the confidence
ax3.plot(times, confidences)
# draw a line at tolerance
ax3.plot(times, [tolerance]*len(confidences))
ax3.axis( xmin = times[0], xmax = times[-1])
ax3.set_ylabel('condidence')
set_xlabels_sample2time(ax3, times[-1], samplerate)
plt.show()
#plt.savefig(os.path.basename(filename) + '.svg')

ModuleNotFoundError: ignored